In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import spacy
from wordcloud import WordCloud
import seaborn as sns

In [ ]:
news247_grel.shape

Φτιάχνω τη λούπα για να πάρω τα λινκς που θέλω για τις ελληνικές βουλευτικές εκλογές του 2023. Το "εκλογές 2023" αποτελεί σχετικά παλιό tag του news247, γι' αυτό το λόγο μου έφερνε- εκτός από τα άρθρα των εκλογών- νέα άρθρα από άλλες κατηγορίες. Για να "φιλτράρω" τα άρθρα που θα μου έρθουν μέσω του κώδικα έβαλα παραμέτρους με if και if not.

In [ ]:
urls = []
base_url = "https://www.news247.gr/ekloges/page/"

for page_num in range(56, 84):
    url = base_url + str(page_num)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('article')

    for article in articles:
        article_url = article.find('a')['href']
        if 'ekloges' in article_url and 'evroekloges' not in article_url and "2024" not in article_url and "kasselakis" not in article_url:
            urls.append(article_url)


print(urls)

Μετατροπή σε dataframe και αποθήκευση στο drive.

In [ ]:
news_247_grel_urls = pd.DataFrame(urls)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
news_247_grel_urls.to_csv("/content/drive/MyDrive/news_247_grel_urls", index = False)

In [ ]:
!ls '/content/drive/My Drive/articles_df'

In [ ]:
news247_grel = pd.read_csv('/content/drive/My Drive/articles_df')

In [ ]:
news247_grel_urls

In [ ]:
news247_grel = news247_grel_urls.rename(columns={"0": 'URLs'}) #μετονομασία στήλης με rename

In [ ]:
news247_grel

Φτιάχνω λίστες για να αποθηκεύσω όσα scrape-άρω και λούπα για τη διαδικασία. Μέσω της  λειτουργίας enumerate, σε συνδυασμό με το index και το url, δίνεται η δυνατότητα να παρακολουθώ ακριβώς το που βρίσκεται ο κώδικάς μου όσον αφορά το scraping. Παρατηρώ ότι οι μεταβλητές "tag", σε συνδυασμό με το else, κάνουν τον κώδικα να εκτελείται γρηγορότερα.

In [ ]:
full_articles_list = []
titles = []



for index, article_url in enumerate(news247_grel['URLs']):

    try:

        response = requests.get(article_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        full_article_dict = {}
        full_article_dict['site'] = "news247.gr"
        full_article_dict['url'] = article_url


        title_tag = soup.find('h1', {"class": "single_article__title article__small_title article__title"})
        title = title_tag.text.strip() if title_tag else 'No Title Found'
        full_article_dict['title'] = title


        full_articles_list.append(full_article_dict)


        print(f"Processed {index + 1}/{len(news247_grel)}: {article_url}")



    except Exception as e:
        print(f"Error processing {article_url}: {e}")
        full_article_dict['title'] = 'Error'
        full_articles_list.append(full_article_dict)


articles_df = pd.DataFrame(full_articles_list)

print(articles_df.head())




In [ ]:
(full_article_dict)

In [ ]:
articles_df = pd.DataFrame(full_articles_list)

In [ ]:
articles_df

Εδώ προσπαθώ να βρω τα υπόλοιπα στοιχεία των άρθρων στην html του news247

In [ ]:
link = "https://www.news247.gr/kosmos/noa-argkamani-eleftheri-i-26xroni-omiros-epanenothike-me-tin-arrosti-mitera-tis/"
response = requests.get(link)
soup = BeautifulSoup(response.text, 'html.parser')
article = soup.find("article")

category = article.find("a")["title"]
author = article.find("span", {"class": "post__author"}).text.strip()
datetime = article.find("li", {"class": "article__date s-font"}).text.strip()
print(category)
print(author)
print(datetime)

In [ ]:
full_articles_list = []


for index, article_url in enumerate(news247_grel['url']):
    try:

        response = requests.get(article_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        full_article_dict = {}
        full_article_dict['site'] = "news247.gr"
        full_article_dict['url'] = article_url


        title_tag = soup.find('h1', {"class": "single_article__title article__small_title article__title"})
        title = title_tag.text.strip() if title_tag else 'No Title Found'
        full_article_dict['title'] = title


        category_tag = soup.find("a", {"title": True})
        category = category_tag['title'].strip() if category_tag else 'No Category Found'
        full_article_dict['category'] = category

        author_tag = soup.find("span", {"class": "post__author"})
        author = author_tag.text.strip() if author_tag else 'No Author Found'
        full_article_dict['author'] = author

        datetime_tag = soup.find("li", {"class": "article__date s-font"})
        datetime = datetime_tag.text.strip() if datetime_tag else 'No Date/Time Found'
        full_article_dict['datetime'] = datetime


        text_tag = soup.find("div", {"class": "paragraph"})
        text = text_tag.text.strip() if text_tag else 'No Text Found'
        full_article_dict['text'] = text


        full_articles_list.append(full_article_dict)


        print(f"Processed {index + 1}/{len(news247_grel)}: {article_url}")


        time.sleep(0.200)

    except Exception as e:
        print(f"Error processing {article_url}: {e}")
        full_article_dict['title'] = 'Error'
        full_article_dict['category'] = 'Error'
        full_article_dict['author'] = 'Error'
        full_article_dict['datetime'] = 'Error'
        full_article_dict['text'] = 'Error'
        full_articles_list.append(full_article_dict)

news247_grel = pd.DataFrame(full_articles_list)

news247_grel.to_csv('news247_grel_articles_with_details.csv', index=False)
news247_grel.to_csv("/content/drive/MyDrive/news247_grel", index = False)


In [ ]:
news247_grel

In [ ]:
news247_grel.to_csv("/content/drive/MyDrive/news247_grel", index = False)

In [ ]:
news247_grel

In [ ]:
full_articles_list = []

for index, article_url in enumerate(news247_grel2_urls['URLs']):
    try:
        response = requests.get(article_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        full_article_dict = {}
        full_article_dict['site'] = "news247.gr"
        full_article_dict['url'] = article_url

        title_tag = soup.find('h1', {"class": "single_article__title article__small_title article__title"})
        title = title_tag.text.strip() if title_tag else 'No Title Found'
        full_article_dict['title'] = title

        category_tag = soup.find("a", {"title": True})
        category = category_tag['title'].strip() if category_tag else 'No Category Found'
        full_article_dict['category'] = category

        author_tag = soup.find("span", {"class": "post__author"})
        author = author_tag.text.strip() if author_tag else 'No Author Found'
        full_article_dict['author'] = author

        datetime_tag = soup.find("li", {"class": "article__date s-font"})
        datetime = datetime_tag.text.strip() if datetime_tag else 'No Date/Time Found'
        full_article_dict['datetime'] = datetime

        summary_tag = soup.find("h2", {"class": "article__lead"})
        summary = summary_tag.text.strip() if summary_tag else 'No Text Found'
        full_article_dict["summary"] = summary

        text_tag = soup.find("div", {"class": "paragraph"})
        text = text_tag.text.strip() if text_tag else 'No Text Found'
        full_article_dict['text'] = text

        full_articles_list.append(full_article_dict)

        print(f" {index + 1}/{len(news247_grel2_urls)}: {article_url}")

        time.sleep(0.200)

    except Exception as e:
        print(f"Error processing {article_url}: {e}")
        full_article_dict = {}
        full_article_dict['site'] = "news247.gr"
        full_article_dict['url'] = article_url
        full_article_dict['title'] = 'Error'
        full_article_dict['category'] = 'Error'
        full_article_dict['author'] = 'Error'
        full_article_dict['datetime'] = 'Error'
        full_article_dict['summary'] = 'Error'
        full_article_dict['text'] = 'Error'
        full_articles_list.append(full_article_dict)

news247_grel = pd.DataFrame(full_articles_list)

news247_grel

In [ ]:
news247_grel.to_csv("/content/drive/MyDrive/news247_grel", index = False)

In [ ]:
news247_grel.dtypes

In [ ]:
news247_grel = pd.read_csv('/content/drive/My Drive/news247_grel')

Εδώ διαπιστώνω ότι έχω ξεχάσει τα summaries...

In [ ]:
summaries_list = []

for index, article_url in enumerate(news247_grel['url']):
    try:
        response = requests.get(article_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        summary_dict = {}
        summary_dict['site'] = "news247.gr"
        summary_dict['url'] = article_url

        summary_tag = soup.find("h2", {"class": "article__lead"})
        summary = summary_tag.text.strip() if summary_tag else 'No Summary Found'
        summary_dict["summary"] = summary

        summaries_list.append(summary_dict)

        print(f"Processed {index + 1}/{len(news247_grel)}: {article_url}")

        time.sleep(0.200)

    except Exception as e:
        print(f"Error processing {article_url}: {e}")
        summaries_list.append({'site': 'news247.gr', 'url': article_url, 'summary': 'Error'})

summaries_df = pd.DataFrame(summaries_list)

news247_grel = pd.merge(news247_grel, summaries_df, on='url', how='left')

print(news247_grel.head())

summaries_list = []

for index, article_url in enumerate(news247_grel['url']):
    try:
        response = requests.get(article_url)
        soup = BeautifulSoup(response.text, 'html.parser')

        summary_dict = {}
        summary_dict['site'] = "news247.gr"
        summary_dict['url'] = article_url


        summary_tag = soup.find("h2", {"class": "article__lead"})
        summary = summary_tag.text.strip() if summary_tag else 'No Summary Found'
        summary_dict["summary"] = summary


        summaries_list.append(summary_dict)

        print(f"Processed {index + 1}/{len(news247_grel)}: {article_url}")

        time.sleep(0.200)

    except Exception as e:
        print(f"Error processing {article_url}: {e}")
        summaries_list.append({'site': 'news247.gr', 'url': article_url, 'summary': 'Error'})

summaries_df = pd.DataFrame(summaries_list)

news247_grel = pd.merge(news247_grel, summaries_df, on='url', how='left')

print(news247_grel.head())


In [ ]:
summaries_df

In [ ]:
summaries_df = summaries_df.drop(['site', 'url'], axis=1)

In [ ]:
summaries_df

In [ ]:
news247_grel = pd.read_csv("/content/drive/MyDrive/news247_grel")

In [ ]:
news247_grel['summary'] = summaries_df['summary']

In [ ]:
news247_grel

In [ ]:
new_order = ['site','url', 'title', 'category', 'author', 'datetime', 'summary', 'text']

news247_grel = news247_grel[new_order]


In [ ]:
news247_grel.to_csv("/content/drive/MyDrive/news247_greek", index = False)

In [ ]:
news247_grel = pd.read_csv('/content/drive/My Drive/news247_grel')

In [ ]:
news247_grel2 =  pd.read_csv('/content/drive/My Drive/news247_grel2')

In [ ]:
news247_greek_elections = pd.concat([news247_grel, news247_grel2], ignore_index=True)

In [ ]:
news247_grel.to_csv('C:\\Users\\stath\\news247_greek_elections', index = False)

Συνεχίζουμε στο news247 greek_elections

In [ ]:
news247_grel = pd.read_csv('C:\\Users\\stath\\news247_greek_elections')

In [ ]:
import pandas as pd

def replace_month_name(date_str):
    month_map = {
        "Ιανουαρίου": "01",
        "Φεβρουαρίου": "02",
        "Μαρτίου": "03",
        "Απριλίου": "04",
        "Μαΐου": "05",
        "Ιουνίου": "06",
        "Ιουλίου": "07",
        "Αυγούστου": "08",
        "Σεπτεμβρίου": "09",
        "Οκτωβρίου": "10",
        "Νοεμβρίου": "11",
        "Δεκεμβρίου": "12"
    }
    
    if isinstance(date_str, str):
        for greek_month, month_num in month_map.items():
            if greek_month in date_str:
                return date_str.replace(greek_month, month_num)
    return date_str

news247_grel.loc[:646, 'datetime'] = news247_grel.loc[:646, 'datetime'].apply(replace_month_name)

news247_grel.loc[:646, 'datetime'] = pd.to_datetime(news247_grel.loc[:646, 'datetime'], format='%d %m %Y %H:%M', errors='coerce')


In [ ]:
news247_grel.drop(news247_grel.index[[876,877]], inplace=True)


In [ ]:
pd.set_option('display.max_rows', 876)

In [ ]:
news247_grel["title"] = news247_grel["title"].str.replace("\n", "")

In [ ]:
news247_grel["text"] = news247_grel["text"].str.replace("\n", "")
news247_grel["text"] = news247_grel["text"].str.replace("\xa0", "")
news247_grel["text"] = news247_grel["text"].str.replace("https tinyurl com", "") 
news247_grel["text"] = news247_grel["text"].str.replace("Ακολουθήστε το News 24/7 στο Google News και μάθετε πρώτοι όλες τις ειδήσεις","")
news247_grel["text"] = news247_grel["text"].str.replace("Εκλογές 2023: Όλες οι εξελίξεις, για την εκλογική αναμέτρησηΠού Ψηφίζω: Μάθε πού ψηφίζεις, βρες το εκλογικό σου διαμέρισμαΑποτελέσματα Εκλογών 2023: Αναλυτικά τα αποτελέσματα με χάρτη επικράτειας και περιφερειών για τις εθνικές εκλογές 2023Pollitics – Όλες οι δημοσκοπήσεις σε μία σελίδα με αναλυτικά γραφήματαLive Εκπομπή εκλογών 2023: Δες τις εκλογές αλλιώςΑκολουθήστε το News 24/7 στο Google News και μάθετε πρώτοι όλες τις ειδήσεις", "")
news247_grel["text"] = news247_grel["text"].str.replace("Εκλογές 2023: Όλες οι εξελίξεις, για την εκλογική αναμέτρησηΠού Ψηφίζω: Μάθε που ψηφίζεις, βρες το εκλογικό σου διαμέρισμαΑποτελέσματα Εκλογών 2023: Αναλυτικά τα αποτελέσματα με χάρτη επικράτειας και περιφερειών για τις εθνικές εκλογές 2023Pollitics – Όλες οι δημοσκοπήσεις σε μία σελίδα με αναλυτικά γραφήματαLive Εκπομπή εκλογών 2023: Δες τις εκλογές αλλιώς","")
news247_grel["text"] = news247_grel["text"].str.replace("Ακολουθήστε το News24/7 στο Google News και μάθετε πρώτοι όλες τις ειδήσεις","")
news247_grel["text"] = news247_grel["text"].str.replace("Ακολουθήστε το News 24/7 στοGoogle Newsκαι μάθετε πρώτοι όλες τιςειδήσεις", "")
news247_grel["text"] = news247_grel["text"].str.replace("Είχαμε κάνει δουλειά για να γίνει αυτό”.Ακολουθήστε το News247.gr στο Google News και μάθετε πρώτοι όλες τις ειδήσεις", "Είχαμε κάνει δουλειά για να γίνει αυτό")
news247_grel["text"] = news247_grel["text"].str.replace("Υγεία.Ακολουθήστε το News247.gr στο Google News και μάθετε πρώτοι όλες τις ειδήσεις", "Υγεία.") 
news247_grel["text"] = news247_grel["text"].str.replace("Μητσοτάκης.Ακολουθήστε το News247.gr στο Google News και μάθετε πρώτοι όλες τις ειδήσεις", "Μητσοτάκης")
news247_grel["text"] = news247_grel["text"].str.replace("Ακολουθήστε το News247.gr στο Google News και μάθετε πρώτοι όλες τις ειδήσεις", "")

In [ ]:
nlp = spacy.load('el_core_news_md')

In [ ]:
news247_grel_full_text = news247_grel['text'].str.cat(sep = ' ')

In [ ]:
print(len(news247_grel_full_text ))

In [ ]:
nlp.max_length = 4000000

In [ ]:
news247_grel_full_doc = nlp(news247_grel_full_text)

In [ ]:
from joblib import dump, load

In [ ]:
#dump(news247_grel_full_doc, 'news247_full_doc.joblib')

In [ ]:
news247_grel_full_doc = load('news247_full_doc.joblib')

In [ ]:
lemmatized_text = ' '.join(token.lemma_ for token in news247_grel_full_doc)

In [ ]:
stopwords = nlp.Defaults.stop_words
stopwords.add("ς")
stopwords.add("μπορώ")
stopwords.add("αναφέρω")
stopwords.add("υπάρχω")
stopwords.add("γίνομαι")
stopwords.add("ή")
stopwords.add("κάνω")
stopwords.add("θέλω")
stopwords.add("κ")
stopwords.add("λέγω")

In [ ]:
lemmatized_text = lemmatized_text.replace("ΣΥΡΙΖΑς", "ΣΥΡΙΖΑ")

wordcloud_news247_grel = WordCloud(
    stopwords = nlp.Defaults.stop_words,
    width = 2000,
    height = 1000,
    background_color = 'black'
 ).generate(lemmatized_text)
fig = plt.figure(
    figsize = (40, 30),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud_news247_grel, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), min_df=0.01, max_df=0.95)

In [ ]:
count_vector = cv.fit_transform(news247_grel["text"])

In [ ]:
news247_grel_results = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=20, ngram_range=(2,2))
count_vector = cv.fit_transform(news247_grel["text"])
news247_grel_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
news247_grel_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=20, ngram_range=(4,10))
count_vector = cv.fit_transform(news247_grel["text"])
news247_grel_polygrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
news247_grel_polygrams.sum(axis =0).sort_values(ascending = False)

In [ ]:


# Assuming `news247_grel` is your DataFrame and 'text' is the column you want to search
search_text = "www"

# Find rows where the 'text' column contains the search_text
matching_rows = news247_grel[news247_grel['text'].str.contains(search_text, na=False)]

# Display the matching rows
print(matching_rows)


In [ ]:
data = {
    "ΣΥΡΙΖΑ ΠΣ"              : 380,
    "αλέξης τσίπρας"          : 337,
    "κυριάκος μητσοτάκης"     : 321,
    "νέα δημοκρατία"         : 270,
    "νέας δημοκρατίας"       : 252,
    "21ης μαΐου"            : 203,
    "αλέξη τσίπρα"           : 192,
    "πρόεδρος ΣΥΡΙΖΑ"        : 190,
    "εκλογές 21ης"           : 176,
    "αρείου πάγου"           : 143,
    "άκρας δεξιάς"           : 140,
    "πρόεδρος ΠΑΣΟΚ"         : 129,
    "μεσαία τάξη"            : 127,
    "τέσσερα χρόνια"         : 124,
    "ΠΑΣΟΚ ΚΙΝΑΛ"            : 120,
    "πρώτης κατοικίας"       : 114,
    "21 μαΐου"               : 111,
    "Νίκος Ανδρουλάκης"      : 107
}

data["Αλέξης Τσίπρας/Αλέξη Τσίπρα"] = data.pop("αλέξης τσίπρας") + data.pop("αλέξη τσίπρα")
data["Νέας Δημοκρατίας/Νέα Δημοκρατία"] = data.pop("νέα δημοκρατία") + data.pop("νέας δημοκρατίας")

df = pd.DataFrame(list(data.items()), columns=['Διγράμματα', 'Αναφορές'])

df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(14, 10), dpi=100)
bars = plt.barh(df['Διγράμματα'], df['Αναφορές'], color='crimson', edgecolor='black')


for bar in bars:
    plt.text(
        bar.get_width() - 5, bar.get_y() + bar.get_height() / 2,
        str(bar.get_width()),
        va='center',
        ha='right', 
        color='white',
        fontsize=12  
    )


plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα')
plt.xlim(0, 700)
plt.title('Διγράμματα Εθνικών Εκλογών 2023 - News247')
plt.gca().invert_yaxis()
plt.tight_layout()  
plt.show()


In [ ]:
data =  {"πρωθυπουργός Κυριάκος Μητσοτάκης" :      117,
"εκλογές 21ης μαΐου"   :                  99,
"ΜέΡΑ25 Συμμαχία Ρήξη" :                  90,
"ΠΑΣΟΚ Κινήματος Αλλαγής" :             89,
"ΣΥΡΙΖΑ προοδευτική συμμαχία"  :           82,
"προστασία πρώτης κατοικίας"  :           72,
"πρόεδρος ΣΥΡΙΖΑ ΠΣ"  :                   71,
"εκλογές 21ης μαϊου"  :                   56,
"πρόεδρος Νέας Δημοκρατίας"  :            47,
"πρόεδρος πασοκ κινήματος"  :             45,
"πρόεδρος πασοκ κιναλ"   :                44,
"εκλογών 21ης μαΐου"  :                   38,
"ΠΑΣΟΚ Κίνημα Αλλαγής" :                  37,
"κυβέρνηση προοδευτικής συνεργασίας" :     35,
"γγ κε κκε"    :                          35,
"κυβερνητικός εκπρόσωπος Άκης Σκέρτσος" : 29,        
"μείωση τιμών ρύθμιση χρεών" : 28}

data["πρόεδρος πασοκ κιναλ/πρόεδρος πασοκ κινήματος"] = data.pop("πρόεδρος πασοκ κινήματος") + data.pop("πρόεδρος πασοκ κιναλ")

df = pd.DataFrame(list(data.items()), columns=['Πολυγράμματα', 'Αναφορές'])

df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(14, 10), dpi=100)
bars = plt.barh(df['Πολυγράμματα'], df['Αναφορές'], color='crimson', edgecolor='black')


for bar in bars:
    plt.text(
        bar.get_width() - 5, bar.get_y() + bar.get_height() / 2,
        str(bar.get_width()),
        va='center',
        ha='right', 
        color='white',
        fontsize=12  
    )


plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα')
plt.xlim(0, 150)
plt.title('Πολυγράμματα Εθνικών Εκλογών 2023 - News247')
plt.gca().invert_yaxis()
plt.tight_layout()  
plt.show()

In [ ]:
filepath = "https://raw.githubusercontent.com/datajour-gr/DataJournalism/main/Bachelor%20Lessons%202023/Lesson%2010/NRC_GREEK_Translated_6_2020.csv"

In [ ]:
emolex_df = pd.read_csv(filepath)

In [ ]:
emolex_df = emolex_df.drop_duplicates(subset=['word'])
emolex_df = emolex_df.dropna()
emolex_df.reset_index(inplace = True, drop=True)

In [ ]:
vec = CountVectorizer(analyzer = 'word', vocabulary = emolex_df.word,
                      lowercase=False, 
                      strip_accents = 'unicode',  
                      stop_words= list(nlp.Defaults.stop_words),
                      ngram_range=(1, 2))

In [ ]:
matrix = vec.fit_transform(news247_grel["text"])
vocab = vec.get_feature_names_out()
wordcount_df = pd.DataFrame(matrix.toarray(), columns=vocab)

In [ ]:
positive_words = emolex_df[emolex_df.Positive == 1]['word']

In [ ]:
negative_words = emolex_df[emolex_df.Negative == 1]['word']

In [ ]:
news247_grel['positive_text'] = wordcount_df[positive_words].sum(axis=1)

In [ ]:
news247_grel['negative_text'] = wordcount_df[negative_words].sum(axis=1)

In [ ]:
news247_grel['pos/neg_text'] = news247_grel['positive_text'] - news247_grel['negative_text']

In [ ]:
matrix = vec.fit_transform(news247_grel["title"])
vocab = vec.get_feature_names_out()
wordcount_df2 = pd.DataFrame(matrix.toarray(), columns=vocab)

In [ ]:
news247_grel['positive_title'] = wordcount_df2[positive_words].sum(axis=1)

In [ ]:
news247_grel['negative_title'] = wordcount_df2[negative_words].sum(axis=1)

In [ ]:
news247_grel['pos/neg_title'] = news247_grel['positive_title'] - news247_grel['negative_title']

In [ ]:
news247_grel['pos/neg_title'].mean()

In [ ]:
short_df = news247_grel[["title", "datetime"]]

In [ ]:
#pd.set_option('display.max_colwidth', None)

In [ ]:
short_df

In [ ]:
phrase_to_remove = "Η άκρα δεξιά και το διακύβευμα των ευρωεκλογών"
news247_grel = news247_grel[~news247_grel['title'].str.contains(phrase_to_remove, na=False)]

In [ ]:
titles_Τσίπρας= news247_grel[news247_grel['title'].str.contains('Τσίπρας', case=False, na=False)]

In [ ]:
row_indices = [7, 8, 36, 39, 40, 58, 69, 85, 95, 96, 98, 111, 112, 126, 131, 133, 152, 166, 169, 182, 183, 192, 194, 204, 206, 211, 220, 224, 225, 227, 240, 263, 271, 287, 299, 300, 318, 319, 320, 321, 327, 334, 335, 338, 339, 344, 353, 354, 357, 358, 372, 373, 379, 383, 395, 412, 425, 430, 440, 443, 447, 462, 466, 467, 468, 480, 486, 487, 510, 520, 526, 531, 548, 556, 562, 568, 572]

news247_Σχόλιο = news247_grel.iloc[row_indices]



In [ ]:
news247_Σχόλιο = news247_grel[[7, 8, 36, 39, 40 ,58, 69,85,95,96, 98,111,112,126,131,133,152,166,169,182,183,192,194,204,206,211,220,224,225,227,240,263,271,287,299,300,318,319,320,321,327,334,335,338,339,344,353,354,357,358,372,373,379,383,395,412,425,430,440,443,447,462,466,467,468,480,486,487,510,520,526,531,548,556,562,568,572]]

In [ ]:
custom_negative_words = {"Θάτσερ","θατσερικά", "Μητσοτακισμός", "καθεστώς","επιτέθηκαν","χάος", "Απομονωμένος","ξεπερασμένα", "γελοίο", "επικίνδυνο","τερατογένεση", "εφιάλτης", "διασύρει", "Dealer","επιβάρυνση","ψέμα","παραλήρημα","παρακράτος", "έγκλημα","μπάχαλο"}

In [ ]:
custom_negative_df = pd.DataFrame({'word': list(custom_negative_words), 'Positive': 0, 'Negative': 1})
emolex_df = pd.concat([emolex_df, custom_negative_df], ignore_index=True).drop_duplicates(subset=['word']).reset_index(drop=True)


In [ ]:
news247_ΝΔ = news247_grel[[6,7,8,16,17,25,36,39,42,49,61,71,81,88,90,103,113,115,120,123,125,129,139,143,144,158,159,163,174,175,176,179,188,201,208,213,230,238,239,247,252,254,255,260,267,269,272,279,284,285,301,305,309,315,324,329,332,333,343,351,352,363,371,380,387,388,390,398,410,416,418,424,427,431,444,445,446,450,451,461,463,477,485,498,502,505,507,515,527,532,536,541,542,549,563,566,571,575]]

In [ ]:
news247_Γενική_Αρθρογραφία = news247_grel[[0,1,2,4,5,9,10,13,19,20,29,30,31,33,34,37,43,47,48,51,52,53,57,59,63,66,68,76,77,78,79,80,82,83,84,86,91,93,94,95,97,104,108,111,124,128,141,155,165,166,172,178,189,192,195,196,199,206,214,215,216,217,218,223,229,244,248,249,253,257,278,308,316,323,326,340,355,356,361,365,368,370,376,378,381,402,406,407,411,432,449,455,460,471,419,457,469,472,473,484,491,509,514,518,528,533,543,545,547,567,569,570],574]

In [ ]:
news247_ΣΥΡΙΖΑ = news247_grel[[3,11,15,21,22,24,27,35,38,40,46,50,54,55,56,60,64,65,67,74,75,99,102,106,107,110,116,118,121,122,127,130,135,136,137,142,148,150,151,156,164,167,173,184,185,187,191,202,209,212,219,221,222,226,228,231,232,234,235,236,243,245,246,250,259,261,262,264,265,268,275,277,286,288,290,291,292,294,298,303,307,310,311,328,330,331,345,347,350,359,364,366,367,374,375,385,386,393,397,399,400,403,405,408,413,417,420,421,426,428,429,433,434,435,436,438,442,453,459,464,470,474,478,481,482,490,492,495,504,508,512,513,517,519,522,534,537,544,546,551,560,564]]

In [ ]:
ΠΑΣΟΚ news247_grel[[18,23,26,32,41,62,70,72,101,109,119,132,138,140,146,154,157,161,171,186,193,198,207,210,211,241,270,273,283,289,295,296,304,306,312,314,320,322,325,342,348,360,369,382,392,404,414,422,437,439,441,454,458,475,480,488,494,496,500,501,506,511,516,524,529,535,540,553,558,565]] 

In [ ]:
ΚΚΕnews247_grel[[14,28,44,73,114,145,149,153,160,200,258,276,282,297,336,337,362,377,389,391,394,396,401,452,456,479,499,525,550,557,559,573]] 

In [ ]:
ΜέΡΑ25news247_grel[[12,45,87,92,105,117,134,147,162,170,177,190,237,242,256,266,274,280,281,293,302,341,346,349,384,415,423,465,476,483,489,497,503,521,523,538,539,552]] 

In [ ]:
Εξωκοινοβουλευτική Αριστερά news247_grel[[89,168,251,313,409,493]] 

In [ ]:
Ελληνική Λύση [180,181,197,203,205,530,531,554,555,561]

In [ ]:
custom_negative_df = pd.DataFrame({'word': list(custom_negative_words), 'Positive': 0, 'Negative': 1})
emolex_df = pd.concat([emolex_df, custom_negative_df]).drop_duplicates(subset=['word']).reset_index(drop=True)

In [ ]:
row_indices = [6,7,8,16,17,25,36,39,42,49,61,71,81,88,90,103,113,115,120,123,125,129,139,143,144,158,159,163,174,175,176,179,188,201,208,213,230,238,239,247,252,254,255,260,267,269,272,279,284,285,301,305,309,315,324,329,332,333,343,351,352,363,371,380,387,388,390,398,410,416,418,424,427,431,444,445,446,450,451,461,463,477,485,498,502,505,507,515,520,527,532,536,541,542,549,563,566,571,575]

news247_ΝΔ = news247_grel.iloc[row_indices]

In [ ]:
row_indices = [3,11,15,21,22,24,27,35,38,40,46,50,54,55,56,60,64,65,67,74,75,99,102,106,107,110,116,118,121,122,127,130,135,136,137,142,148,150,151,156,164,167,173,184,185,187,191,202,209,212,219,221,222,226,228,231,232,234,235,236,243,245,246,250,259,261,262,264,265,268,275,277,286,288,290,291,292,294,298,303,307,310,311,328,330,331,345,347,350,359,364,366,367,374,375,385,386,393,397,399,400,403,405,408,413,417,420,421,426,428,429,433,434,435,436,438,442,453,459,464,470,474,478,481,482,490,492,495,504,508,512,513,517,519,522,534,537,544,546,551,560,564]
news247_ΣΥΡΙΖΑ = news247_grel.iloc[row_indices]

In [ ]:
news247_ΝΔ["pos/neg_text"].mean()

In [ ]:
row_indices = [7, 8, 36, 39, 40 ,58, 69,85,95,96, 98,111,112,126,131,133,152,166,169,182,183,192,194,204,206,211,220,224,225,227,240,263,271,287,299,300,318,319,320,321,327,334,335,338,339,344,353,354,357,358,372,373,379,383,395,412,425,430,440,443,447,462,466,467,468,480,486,487,510,520,526,531,548,556,562,568,572]
news247_Σχόλιο = news247_grel.iloc[row_indices]

In [ ]:
pd.set_option('display.max_colwidth', None)

# Display the full titles
print(news247_Σχόλιο["title"])

In [ ]:
row_indices = [6,7,8,16,17,25,36,39,42,49,58,61,69,71,81,85,88,90,96,98,103,112,113,115,120,123,125,126,129,131,133,139,143,144,158,159,163,169,174,175,176,179,183,188,194,201,208,213,220,224,225,227,240,263,271,287,300,230,238,239,247,252,254,255,260,267,269,272,279,284,285,301,305,309,315,319,324,327,329,332,333,334,338,343,351,352,353,354,363,371,372,373,379,380,387,388,390,395,398,410,412,416,418,424,427,430,431,443,444,445,446,447,450,451,461,462,463,467,468,477,485,487,498,502,505,507,510,515,526,527,532,536,541,542,549,562,563,566,568,571,575]

news247_ΝΔ = news247_grel.iloc[row_indices]

In [ ]:
news247_ΝΔ.shape

In [ ]:
row_indices = [3,11,15,21,22,24,27,35,38,40,46,50,54,55,56,60,64,65,67,74,75,99,102,106,107,110,116,118,121,122,127,130,135,136,137,142,148,150,151,156,164,167,173,182,184,185,187,191,202,209,212,219,221,222,226,228,231,232,234,235,236,243,245,246,250,259,261,262,264,265,268,275,277,286,288,290,291,292,294,298,303,307,310,311,318,328,330,331,339,344,345,347,350,357,359,364,366,367,374,375,385,386,393,397,399,400,403,405,408,413,417,420,421,426,428,429,433,434,435,436,438,440,442,453,459,464,466,470,474,478,481,482,490,492,495,504,508,512,513,517,519,522,534,537,544,546,548,551,560,564]
news247_ΣΥΡΙΖΑ = news247_grel.iloc[row_indices]

In [ ]:
news247_ΣΥΡΙΖΑ.shape

In [ ]:
news247_Άλλο = news247_grel[[0,1,2,4,5,9,10,13,19,20,29,30,31,33,34,37,43,47,48,51,52,53,57,59,63,66,68,76,77,78,79,80,82,83,84,86,91,93,94,95,97,104,108,111,124,128,141,155,165,166,172,178,189,192,195,196,199,204,206,214,215,216,217,218,223,229,244,248,249,253,257,278,308,316,321,323,326,335,340,355,356,361,365,368,370,376,378,381,402,406,407,411,425,432,449,455,460,471,419,457,469,472,473,484,491,509,514,518,528,533,543,545,547,567,569,570,574]

In [ ]:
Ακροδεξιά[358,383,486,556,572]

In [ ]:
news247_Σχόλιο.shape

In [ ]:

news247_grel['κόμματα'] = ''

news247_grel.loc[[6,7,8,16,17,25,36,39,42,49,58,61,69,71,81,85,88,90,96,98,103,112,113,115,120,123,125,126,129,131,133,139,143,144,158,159,163,169,174,175,176,179,183,188,194,201,208,213,220,224,225,227,240,263,271,287,300,230,238,239,247,252,254,255,260,267,269,272,279,284,285,301,305,309,315,319,324,327,329,332,333,334,338,343,351,352,353,354,363,371,372,373,379,380,387,388,390,395,398,410,412,416,418,424,427,430,431,443,444,445,446,447,450,451,461,462,463,467,468,477,485,487,498,502,505,507,510,515,526,527,532,536,541,542,549,562,563,566,568,571,575], 'κόμματα'] = 'ΝΔ'

news247_grel.loc[[3,11,15,21,22,24,27,35,38,40,46,50,54,55,56,60,64,65,67,74,75,99,102,106,107,110,116,118,121,122,127,130,135,136,137,142,148,150,151,156,164,167,173,182,184,185,187,191,202,209,212,219,221,222,226,228,231,232,234,235,236,243,245,246,250,259,261,262,264,265,268,275,277,286,288,290,291,292,294,298,303,307,310,311,318,328,330,331,339,344,345,347,350,357,359,364,366,367,374,375,385,386,393,397,399,400,403,405,408,413,417,420,421,426,428,429,433,434,435,436,438,440,442,453,459,464,466,470,474,478,481,482,490,492,495,504,508,512,513,517,519,522,534,537,544,546,548,551,560,564], 'κόμματα'] = 'ΣΥΡΙΖΑ'

news247_grel.loc[[358,383,486,556,572], 'κόμματα'] = 'Ακροδεξιά'

news247_grel.loc[[0,1,2,4,5,9,10,13,19,20,29,30,31,33,34,37,43,47,48,51,52,53,57,59,63,66,68,76,77,78,79,80,82,83,84,86,91,93,94,95,97,104,108,111,124,128,141,155,165,166,172,178,189,192,195,196,199,204,206,214,215,216,217,218,223,229,244,248,249,253,257,278,308,316,321,323,326,335,340,355,356,361,365,368,370,376,378,381,402,406,407,411,425,432,449,455,460,471,419,457,469,472,473,484,491,509,514,518,528,533,543,545,547,567,569,570,574], 'κόμματα'] = 'Γενική Αρθρογραφία'

news247_grel.loc[[180,181,197,203,205,530,531,554,555,561], 'κόμματα'] = 'Ελληνική Λύση'

news247_grel.loc[[18,23,26,32,41,62,70,72,101,109,119,132,138,140,146,154,157,161,171,186,193,198,207,210,211,241,270,273,283,289,295,296,304,306,312,314,320,322,325,342,348,360,369,382,392,404,414,422,437,439,441,454,458,475,480,488,494,496,500,501,506,511,516,524,529,535,540,553,558,565], 'κόμματα'] = 'ΠΑΣΟΚ'

news247_grel.loc[[14,28,44,73,114,145,149,153,160,200,258,276,282,297,336,337,362,377,389,391,394,396,401,452,456,479,499,525,550,557,559,573], 'κόμματα'] = 'ΚΚΕ'

news247_grel.loc[[12,45,87,92,105,117,134,147,162,170,177,190,237,242,256,266,274,280,281,293,302,341,346,349,384,415,423,465,476,483,489,497,503,521,523,538,539,552], 'κόμματα'] = 'ΜέΡΑ25'

news247_grel.loc[[89,168,251,313,409,493], 'κόμματα'] = 'Εξωκοινοβουλευτική Αριστερά'


In [ ]:
news247_Γενική_Αρθρογραφία = news247_grel.loc[news247_grel['κόμματα'] == 'Γενική Αρθρογραφία']
news247_ΝΔ = news247_grel.loc[news247_grel['κόμματα'] == 'ΝΔ']
news247_ΣΥΡΙΖΑ = news247_grel.loc[news247_grel['κόμματα'] == 'ΣΥΡΙΖΑ']
news247_ΠΑΣΟΚ = news247_grel.loc[news247_grel['κόμματα'] == 'ΠΑΣΟΚ']
news247_ΚΚΕ = news247_grel.loc[news247_grel['κόμματα'] == 'ΚΚΕ']
news247_ΜέΡΑ25 = news247_grel.loc[news247_grel['κόμματα'] == 'ΜέΡΑ25']
news247_Ακροδεξιά = news247_grel.loc[news247_grel['κόμματα'] == 'Ακροδεξιά']
news247_Ελληνική_Λύση = news247_grel.loc[news247_grel['κόμματα'] == 'Ελληνική Λύση']
news247_Εξωκοινοβουλευτική_Αριστερά = news247_grel.loc[news247_grel['κόμματα'] == 'Εξωκοινοβουλευτική Αριστερά']

In [ ]:
news247_grel = news247_grel.drop(index=range(577, 849))


In [ ]:
news247_Σχόλιο.loc[[7,8,36,39,58,69,85,96,98,112,126,131,133,152,169,182,183,194,220,224,225,227,240,263,271,287,299,300,319,327,334,338,344,353,372,373,379,395,412,430,447,462,468,487,510,526,562,568], 'κόμματα_2'] = 'ΝΔ'
news247_Σχόλιο.loc[[40,111,318,339,354,357,440,443,467,548],'κόμματα_2'] = 'ΣΥΡΙΖΑ'
news247_Σχόλιο.loc[[95,166,192,204,206,321,335,425,466], 'κόμματα_2'] = 'Γενική Αρθρογραφία'
news247_Σχόλιο.loc[[211,320,480], 'κόμματα_2'] = 'ΠΑΣΟΚ'
news247_Σχόλιο.loc[[358,383,486,556,572], 'κόμματα_2'] = 'Ακροδεξιά'
news247_Σχόλιο.loc[[520], 'κόμματα_2'] = 'ΚΚΕ'
news247_Σχόλιο.loc[[531], 'κόμματα_2'] = 'Ελληνική Λύση'


In [ ]:
news247_Σχόλιο

In [ ]:
news247_ΝΔ

In [ ]:
news247_Γενική_Αρθρογραφία = news247_grel.loc[news247_grel['κόμματα'] == 'Γενική Αρθρογραφία']
news247_ΝΔ = news247_grel.loc[news247_grel['κόμματα'] == 'ΝΔ']
news247_ΣΥΡΙΖΑ = news247_grel.loc[news247_grel['κόμματα'] == 'ΣΥΡΙΖΑ']
news247_ΠΑΣΟΚ = news247_grel.loc[news247_grel['κόμματα'] == 'ΠΑΣΟΚ']
news247_ΚΚΕ = news247_grel.loc[news247_grel['κόμματα'] == 'ΚΚΕ']
news247_ΜέΡΑ25 = news247_grel.loc[news247_grel['κόμματα'] == 'ΜέΡΑ25']
news247_Ακροδεξιά = news247_grel.loc[news247_grel['κόμματα'] == 'Ακροδεξιά']
news247_Ελληνική_Λύση = news247_grel.loc[news247_grel['κόμματα'] == 'Ελληνική Λύση']
news247_Εξωκοινοβουλευτική_Αριστερά = news247_grel.loc[news247_grel['κόμματα'] == 'Εξωκοινοβουλευτική Αριστερά']

In [ ]:
 news247_ΝΔ["pos/neg_text"].mean()

In [ ]:
 news247_ΝΔ["pos/neg_title"].mean()

In [ ]:
news247_Γενική_Αρθρογραφία["pos/neg_text"].mean()

In [ ]:
news247_Γενική_Αρθρογραφία["pos/neg_title"].mean()

In [ ]:
news247_ΣΥΡΙΖΑ["pos/neg_text"].mean()

In [ ]:
news247_ΣΥΡΙΖΑ["pos/neg_title"].mean()

In [ ]:
news247_ΠΑΣΟΚ["pos/neg_text"].mean()

In [ ]:
news247_ΠΑΣΟΚ["pos/neg_title"].mean()

In [ ]:
news247_ΚΚΕ["pos/neg_text"].mean()

In [ ]:
news247_ΚΚΕ["pos/neg_title"].mean()

In [ ]:
news247_ΜέΡΑ25["pos/neg_text"].mean()

In [ ]:
news247_ΜέΡΑ25["pos/neg_title"].mean()

In [ ]:
news247_Ακροδεξιά["pos/neg_text"].mean()

In [ ]:
news247_Ακροδεξιά["pos/neg_title"].mean()

In [ ]:
news247_Ελληνική_Λύση["pos/neg_text"].mean()

In [ ]:
news247_Ελληνική_Λύση["pos/neg_title"].mean()

In [ ]:
news247_Εξωκοινοβουλευτική_Αριστερά["pos/neg_text"].mean()

In [ ]:
news247_Εξωκοινοβουλευτική_Αριστερά["pos/neg_title"].mean()

In [ ]:
news247_grel['κόμματα'].value_counts(ascending = False)

In [ ]:
ΣΥΡΙΖΑ                         
ΝΔ                             
Γενική Αρθρογραφία             
ΠΑΣΟΚ                           
ΜέΡΑ25                          
ΚΚΕ                             
Ελληνική Λύση                   
Εξωκοινοβουλευτική Αριστερά      

In [ ]:
from pywaffle import Waffle

parties = ['Νέα Δημοκρατία', 'ΣΥΡΙΖΑ', 'ΠΑΣΟΚ', 'ΚΚΕ', 'ΜέΡΑ25', 'Ακροδεξιά', 'Ελληνική Λύση', 'Γενική Αρθρογραφία', 'Εξωκοινοβουλευτική Αριστερά']
post_counts = [141, 150, 70, 32, 38, 5, 10, 117, 6]

total_posts = sum(post_counts)
percentages = [count / total_posts * 100 for count in post_counts]

colors = ['blue', 'mediumpurple', 'green', 'red', 'darkorange', 'darkseagreen', 'lightsteelblue', 'grey', 'fuchsia']

data = {f"{party} ({count}, {percentage:.1f}%)": count for party, count, percentage in zip(parties, post_counts, percentages)}

fig = plt.figure(
    FigureClass=Waffle, 
    rows=18, 
    values=data, 
    colors=colors, 
    legend={'loc': 'upper left', 'bbox_to_anchor': (1, 1), 'fontsize': 10, 'title': 'Κόμματα'},
    title={'label': 'Σύνολο και Ποσοστά δημοσιευμάτων για τα κόμματα - News247', 'loc': 'center', 'fontsize': 16},
    block_arranging_style='snake', 
    figsize=(14, 12)
)

plt.show()

In [ ]:
means = {
    'ΝΔ': {
        'Πολικότητα Κειμένων': 1.73,
        'Πολικότητα Τίτλων': -0.007
    },
    'ΣΥΡΙΖΑ': {
        'Πολικότητα Κειμένων': 1.14,
        'Πολικότητα Τίτλων': - 0.05
    },
    'ΠΑΣΟΚ': {
        'Πολικότητα Κειμένων': 2.75,
        'Πολικότητα Τίτλων': 0.05
    },
    'ΚΚΕ': {
        'Πολικότητα Κειμένων': - 0.09,
        'Πολικότητα Τίτλων': - 0.06
    },
    'ΜέΡΑ25': {
        'Πολικότητα Κειμένων': - 3.05,
        'Πολικότητα Τίτλων': - 0.31
    } }
    

means_df = pd.DataFrame(means).T 


color_map = {
    'ΝΔ': 'blue',
    'ΣΥΡΙΖΑ': 'mediumpurple',
    'ΠΑΣΟΚ': 'green',
    'ΚΚΕ': 'red',
    'ΜέΡΑ25': 'darkorange',
    
}

new_col_titles = ['Πολικότητα Κειμένων', 'Πολικότητα Τίτλων']


fig, ax = plt.subplots(figsize=(14, 8))

ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.set_frame_on(False)

table = ax.table(cellText=means_df.values,
                 rowLabels=means_df.index,
                 colLabels=new_col_titles,
                 cellLoc='center',
                 loc='center',
                 colColours=['lightgrey'] * len(new_col_titles),
                 rowColours=['white'] * len(means_df.index))  


for i, label in enumerate(means_df.index):
    row_color = color_map.get(label, 'white')  
    for j in range(len(new_col_titles)):
        table[(i + 1, j)].set_facecolor(row_color)  

table.auto_set_font_size(False)
table.set_fontsize(14)  
table.scale(1.5,3)

In [ ]:
news247_ΝΔ_full_text = news247_ΝΔ['text'].str.cat(sep = ' ')

In [ ]:
news247_ΝΔ_full_doc = nlp(news247_ΝΔ_full_text)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=40, ngram_range=(2,5))
count_vector = cv.fit_transform(news247_ΝΔ["text"])
news247_ΝΔ_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
news247_ΝΔ_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


data = {
    "κυριάκος μητσοτάκης": 153,
    "νέα δημοκρατία": 89,
    "νέας δημοκρατίας": 81,
    "πρωθυπουργός κυριάκος μητσοτάκης": 59,
    "εκλογές 21ης Μαΐου": 81,
    "κυβερνητικός εκπρόσωπος": 38,
    "εκ νέου": 34,
    "δισ ευρώ": 32,
    "Aλέξη Tσίπρα": 31,
    "τέσσερα χρόνια": 30,
    "κυριάκου μητσοτάκη": 28,
    "κυβέρνηση ηττημένων": 28,
    "δεύτερες εκλογές": 23,
    "κυβερνητικός εκπρόσωπος άκης σκέρτσος": 22,
    "Mητσοτάκης δήλωσε": 21,
    "Mητσοτάκης υποστήριξε": 21,
    "πρωθυπουργός πρόεδρος Νέας Δημοκρατίας": 20,
    "ΠΑΣΟΚ Ανδρουλάκη": 18,
    "κάλπη 21ης": 18,
}


data["Νέα Δημοκρατία/Νέας Δημοκρατίας"] = data.pop("νέας δημοκρατίας") + data.pop("νέα δημοκρατία")
data["Κυριάκος Μητσοτάκης/Κυριάκου Μητσοτάκη"] = data.pop("κυριάκος μητσοτάκης") + data.pop("κυριάκου μητσοτάκη")

df = pd.DataFrame(list(data.items()), columns=["Διγράμματα/Πολυγράμματα", "Αναφορές"])
df = df.sort_values(by="Αναφορές", ascending=False)


plt.figure(figsize=(10, 8))
bars = plt.barh(df["Διγράμματα/Πολυγράμματα"], df["Αναφορές"], color="blue", edgecolor="crimson")


for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,
        str(bar.get_width()),
        va="center",
        ha="left",
        color="black",
        fontsize=10
    )

plt.xlabel("Αναφορές")
plt.ylabel("Διγράμματα/Πολυγράμματα")
plt.xlim(0, 200)
plt.title("Διγράμματα/Πολυγράμματα Εθνικών Εκλογών 2023- Νέα Δημοκρατία - News247")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
news247_ΣΥΡΙΖΑ_full_text = news247_ΣΥΡΙΖΑ['text'].str.cat(sep = ' ')

In [ ]:
news247_ΣΥΡΙΖΑ_full_doc = nlp(news247_ΣΥΡΙΖΑ_full_text)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=40, ngram_range=(2,5))
count_vector = cv.fit_transform(news247_ΣΥΡΙΖΑ["text"])
news247_ΣΥΡΙΖΑ_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
news247_ΣΥΡΙΖΑ_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
data = {"συριζα πς" :                     212,
"αλέξης τσίπρας"  :               171,
"πρόεδρος συριζα"  :              102,
"αλέξη τσίπρα" :                    77,
"προοδευτική κυβέρνηση"  :         57,
"μεσαία τάξη"   :                  55,
"συριζα προοδευτική συμμαχία" :             55,
"Κυριάκο Μητσοτάκη/Κυριάκου Μητσοτάκη" :              53,
"νίκη ΣΥΡΙΖΑ"   :                  47,
"πρόεδρος ΣΥΡΙΖΑ ΠΣ"   :           46,
"τέσσερα χρόνια"  :                44,
"πρώτης κατοικίας" :               35,
"ρύθμιση χρεών" :                  32,
"ελληνικός λαός"  :                31,
"πρόγραμμα ΣΥΡΙΖΑ"   :             30,
"συμβόλαιο αλλαγής"  :             29,
"21 μαΐου"   :                     28,
"μείωση φπα"  :                    28,
"Πόπη Τσαπανίδου" :                27,
"κυβέρνηση Μητσοτάκη" :            27,
"μείωση τιμών"  :                   27,
"21ης μαΐου"  :                    27,
"Tσίπρας είπε"  :                  25,
"21 μάη"  :                        25,

"απλή αναλογική"  :                25}


data["ΣΥΡΙΖΑ ΠΣ/ ΣΥΡΙΖΑ Προοδευτική Συμμαχία"] = data.pop("συριζα πς") + data.pop("συριζα προοδευτική συμμαχία")
data["Αλέξης Τσίπρας/Αλέξη Τσίπρα"] = data.pop("αλέξης τσίπρας") + data.pop("αλέξη τσίπρα")

df = pd.DataFrame(list(data.items()), columns=['Διγράμματα', 'Αναφορές'])
df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))

bars = plt.barh(df['Διγράμματα'], df['Αναφορές'], color= 'mediumpurple', edgecolor= 'crimson')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='black',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα/Πολυγράμματα')
plt.xlim(0,350)
plt.title('Διγράμματα/Πολυγράμματα Εθνικών Εκλογών 2023- ΣΥΡΙΖΑ ΠΣ - News247')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
news247_ΠΑΣΟΚ_full_text = news247_ΠΑΣΟΚ['text'].str.cat(sep = ' ')

In [ ]:
news247_ΠΑΣΟΚ_full_doc = nlp(news247_ΠΑΣΟΚ_full_text)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=40, ngram_range=(2,5))
count_vector = cv.fit_transform(news247_ΠΑΣΟΚ["text"])
news247_ΠΑΣΟΚ_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out()) 

In [ ]:
news247_ΠΑΣΟΚ_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
data = {"Πρόεδρος ΠΑΣΟΚ" :                              77,
"ΠΑΣΟΚ Κινήματος Αλλαγής" :                          66,
"νίκος ανδρουλάκης"   :                         62,
"ΠΑΣΟΚ ΚΙΝΑΛ"     :                            58,
"νέα δημοκρατία"  :                             34,
"Πρόεδρος ΠΑΣΟΚ Κινήματος Αλλαγής" :           55,
"νέας δημοκρατίας" :                             27,
"Ανδρέα Παπανδρέου"  :                          24,
"ελληνικού λαού" :                              24,
"21 μαΐου"   :                                  22,
"ελληνικό λαό" :                                19,
"αλέξη τσίπρα"   :                              17,
"νίκου ανδρουλάκη" :                            16,
"Ανδρέας Παπανδρέου":                           15,
"ΝΔ ΣΥΡΙΖΑ"      :                              14,
"νέα γενιά"    :                                14,
'Χρυσή Αυγή'    :                               14,
"ευρωπαϊκή χώρα"  :                             13,
"εκπρόσωπος τύπου"   :                          13,
"δημοκρατική παράταξη" :                        13,
"Κυριάκο Μητσοτάκη" :                           12,
"κοινής αποδοχής"  :                            12,
"πρώτης κατοικίας" :                            12 }


data["Νέα Δημοκρατία/Νέας Δημοκρατίας"] = data.pop("νέας δημοκρατίας") + data.pop("νέα δημοκρατία")
data["Νίκος Ανδρουλάκης/Νίκου Ανδρουλάκη"] = data.pop("νίκος ανδρουλάκης") + data.pop("νίκου ανδρουλάκη")
data["ελληνικού λαού / ελληνικό λαό"] = data.pop("ελληνικού λαού") + data.pop("ελληνικό λαό")
data["Ανδρέα Παπανδρέου / Ανδρέας Παπανδρέου"] = data.pop("Ανδρέα Παπανδρέου") + data.pop("Ανδρέας Παπανδρέου")

df = pd.DataFrame(list(data.items()), columns=["Διγράμματα", "Αναφορές"])
df = df.sort_values(by="Αναφορές", ascending=False)

plt.figure(figsize=(10, 8))
bars = plt.barh(df["Διγράμματα"], df["Αναφορές"], color="green", edgecolor="crimson")

for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,
        str(bar.get_width()),
        va="center",
        ha="left",
        color="black",
        fontsize=10
    )

plt.xlabel("Αναφορές")
plt.ylabel("Διγράμματα/Πολυγράμματα")
plt.xlim(0, 100)
plt.title("Διγράμματα/Πολυγράμματα Εθνικών Εκλογών 2023- ΠΑΣΟΚ-ΚΙΝΑΛ - News247")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
news247_ΜέΡΑ25_full_text = news247_ΜέΡΑ25['text'].str.cat(sep = ' ')

In [ ]:
news247_ΜέΡΑ25_full_doc = nlp(news247_ΜέΡΑ25_full_text)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=30, ngram_range=(2,5))
count_vector = cv.fit_transform(news247_ΜέΡΑ25["text"])
news247_ΜέΡΑ25_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out())

In [ ]:
news247_ΜέΡΑ25_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
data = {
"ΜέΡΑ25 συμμαχία ρήξη"  :                    81,   
"Γιάνης ΒαρουφάκηςΓιάνη Βαρουφάκη" :                         49,
"κόκκινα δάνεια/κόκκινων δανείων" :                   22,
"επικεφαλής ΜέΡΑ25 συμμαχία ρήξη" :           21,
"σχέδιο Δήμητρα"   :                         14,
"Αλέξη Τσίπρα"  :                            13,
"επικεφαλής ΜέΡΑ25"  :                       12,
"ΣΥΡΙΖΑ ΠΑΣΟΚ" :                      11,
"Μητσοτάκης ΑΕ"  :                    10,
"εκπρόσωπος τύπου" :                          9,
"αρπακτικά ταμεία"  :                  8,
"100 ημερών" :                         8,
"ΝΔ ΣΥΡΙΖΑ" :                         8,                         
"χρηματιστηρίου ενέργειας" :            8,
"αγωνιστικής αριστεράς" :              8,
"απλή αναλογική"  :                           8,
"έξοδο ευρώ"  :                              7,
"ψήφο ανοχής"  :                              7,
  
}

df = pd.DataFrame(list(data.items()), columns=['Διγράμματα/Πολυγράμματα', 'Αναφορές'])
df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))

bars = plt.barh(df['Διγράμματα/Πολυγράμματα'], df['Αναφορές'], color= 'orange', edgecolor= 'crimson')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='black',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα/Πολυγράμματα')
plt.xlim(0,100)
plt.title('Διγράμματα/Πολυγράμματα Εθνικών Εκλογών 2023- ΜέΡΑ25 - News247')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
news247_ΚΚΕ_full_text = news247_ΚΚΕ['text'].str.cat(sep = ' ')

In [ ]:
news247_ΚΚΕ_full_doc = nlp(news247_ΚΚΕ_full_text)

In [ ]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), max_features=30, ngram_range=(2,5))
count_vector = cv.fit_transform(news247_ΚΚΕ["text"])
news247_ΚΚΕ_bigrams = pd.DataFrame(count_vector.toarray(), columns=cv.get_feature_names_out())

In [ ]:
news247_ΚΚΕ_bigrams.sum(axis =0).sort_values(ascending = False)

In [ ]:
data = {"Δημήτρης Κουτσούμπας"  :            33,
"ΚΕ ΚΚΕ" :                           31,
"ΝΔ ΣΥΡΙΖΑ"   :                      29,
"ΣΥΡΙΖΑ ΠΑΣΟΚ"  :                    27,
"γγ κε ΚΚΕ Δημήτρης Κουτσούμπας" :   25,                       
"μέλος ΔΣ" :                         22,
"δυνατό ΚΚΕ"  :                    20,
"ΝΔ ΣΥΡΙΖΑ ΠΑΣΟΚ" :                 20,
"είπε Κουτσούμπας" :                 14,
"ΝΔ ΠΑΣΟΚ" :                         12,
"ισχυρό ΚΚΕ" :                       10,
"ιατρικού συλλόγου" :                 9,
"αντιλαϊκή κυβέρνηση"  :             9,
"δυνατός λαός"  :                     9}

df = pd.DataFrame(list(data.items()), columns=['Διγράμματα/Πολυγράμματα', 'Αναφορές'])
df = df.sort_values(by='Αναφορές', ascending=False)

plt.figure(figsize=(10, 8))

bars = plt.barh(df['Διγράμματα/Πολυγράμματα'], df['Αναφορές'], color= 'red', edgecolor= 'crimson')
for bar in bars:
    plt.text(
        bar.get_width() + 2, bar.get_y() + bar.get_height() / 2,  
        str(bar.get_width()),
        va='center',
        ha='left',  
        color='black',
        fontsize=10
    )
    
plt.xlabel('Αναφορές')
plt.ylabel('Διγράμματα/Πολυγράμματα')
plt.xlim(0,50)
plt.title('Διγράμματα/Πολυγράμματα Εθνικών Εκλογών 2023- ΚΚΕ - News247')
plt.gca().invert_yaxis() 
plt.show()

In [ ]:
import re

frames = {
    'Σύγκρουση': [
        'κόντρα', 'VS', 'μάχη', 'επίθεση', 'αντιπαράθεση', '«σφάχτηκαν»', '«επιτίθεται»', "ή", "δίλημμα",
        '«πυρά»', '«Πυρά»','πυρά', 'Πυρά', 'μπηχτή', 'μεταξύ', 'σύγκρουση', '«σκοτωμός»', 'απαντά', '«σπόντα»',
        '«μάχης»', 'μια πλευρά', 'Μητσοτάκη-Τσίπρα', 'απάντηση', 'καυγά', 'αίμα', 'απαντάμε',
        'διαφορά', 'έναντι', 'σύγκριση', 'κατά', 'εναντίον','προβάδισμα', 'Μητσοτάκης-Τσίπρας', 'ΝΔ-ΣΥΡΙΖΑ'
    ]
}

def detect_conflict_frames(title, frames):
    conflict_keywords = frames['Σύγκρουση']
    
    pattern = r'\b(?:' + '|'.join(map(re.escape, conflict_keywords)) + r')\b'

    return bool(re.search(pattern, title))


news247_grel['conflict'] = news247_grel['title'].apply(lambda x: 1 if detect_conflict_frames(x, frames) else 0)

In [ ]:
news247_grel['Conflict'].value_counts()

In [ ]:
6.58